In [1]:
!pip install flask opencv-python mediapipe pyngrok

# Import necessary libraries
from flask import Flask, render_template_string, Response
import cv2
import mediapipe as mp
from pyngrok import ngrok

# Initialize the Flask app
app = Flask(__name__)
camera = None

# MediaPipe face detection setup
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Flag to enable/disable face detection
face_detection_enabled = True

# Generate frames from the camera and apply face detection if enabled
def gen_frames():
    global camera
    global face_detection_enabled
    with mp_face_detection.FaceDetection(min_detection_confidence=0.2) as face_detection:
        camera = cv2.VideoCapture(0)  # Open the default camera
        while True:
            success, frame = camera.read()  # Read frame from camera
            if not success:
                break
            else:
                if face_detection_enabled:
                    # Convert frame to RGB (MediaPipe requires RGB format)
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = face_detection.process(frame_rgb)

                    # Draw face detection results on the frame
                    if results.detections:
                        for detection in results.detections:
                            mp_drawing.draw_detection(frame, detection)

                # Encode frame as JPEG
                ret, buffer = cv2.imencode('.jpg', frame)
                frame = buffer.tobytes()
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

# Flask route for the homepage
@app.route('/')
def index():
    # Using render_template_string to serve the HTML directly
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Camera Control with Face Detection</title>
        </head>
        <body>
            <h1>Camera Control Interface</h1>

            <!-- Video Stream -->
            <h3>Live Camera Feed</h3>
            <img src="{{ url_for('video_feed') }}" width="640" height="480">

            <!-- Buttons to control camera and face detection -->
            <div>
                <button onclick="turnCameraOn()">Turn Camera ON</button>
                <button onclick="turnCameraOff()">Turn Camera OFF</button>
                <button onclick="toggleFaceDetection()">Toggle Face Detection</button>
            </div>

            <!-- JavaScript for button actions -->
            <script>
                function turnCameraOn() {
                    fetch('/camera_on')
                        .then(response => response.text())
                        .then(data => alert(data));
                }

                function turnCameraOff() {
                    fetch('/camera_off')
                        .then(response => response.text())
                        .then(data => alert(data));
                }

                function toggleFaceDetection() {
                    fetch('/toggle_face_detection')
                        .then(response => response.text())
                        .then(data => alert(data));
                }
            </script>
        </body>
        </html>
    ''')

# Flask route for the video feed
@app.route('/video_feed')
def video_feed():
    global camera
    if camera is not None and camera.isOpened():
        return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')
    else:
        return "Camera is not on", 400

# Flask route to turn the camera on
@app.route('/camera_on')
def camera_on():
    global camera
    if camera is None or not camera.isOpened():
        camera = cv2.VideoCapture(0)  # Open the default camera
    if camera.isOpened():
        return "Camera turned ON"
    else:
        return "Failed to turn ON the camera", 500

# Flask route to turn the camera off
@app.route('/camera_off')
def camera_off():
    global camera
    if camera is not None and camera.isOpened():
        camera.release()  # Release the camera
        camera = None
        return "Camera turned OFF"
    return "Camera is already off", 400

# Flask route to toggle face detection on/off
@app.route('/toggle_face_detection')
def toggle_face_detection():
    global face_detection_enabled
    face_detection_enabled = not face_detection_enabled  # Toggle the boolean
    return f"Face detection {'enabled' if face_detection_enabled else 'disabled'}"

# Start the Flask app and ngrok tunnel
if __name__ == "__main__":
    # Set up a tunnel using ngrok to access the Flask app from a public URL
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel: {public_url}")
    app.run()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


ERROR:pyngrok.process.ngrok:t=2024-09-05T01:40:20+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-09-05T01:40:20+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-09-05T01:40:20+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.